In [6]:
import json

import pandas as pd


def read_tokens_df() -> pd.DataFrame:
    with open("../data/external/PII43k_original.jsonl", "r") as f:
        data = [json.loads(line) for line in f.readlines()]
    
    filtered_data = []
    for chunk in data:
        if len(chunk["tokenised_unmasked_text"]) == len(chunk["token_entity_labels"]):
            filtered_data.append(chunk)
    
    print(f"Filtered {len(data) - len(filtered_data)} chunks")
    
    
    for idx, chunk in enumerate(filtered_data):
        chunk["document"] = idx

    df = pd.DataFrame.from_records(
        filtered_data, columns=["document", "tokenised_unmasked_text", "token_entity_labels"]
    )
    df.rename({"tokenised_unmasked_text": "tokens", "token_entity_labels": "labels"}, axis=1, inplace=True)
    token_loc_in_doc = df["tokens"].apply(lambda x: list(range(len(x))))
    df.insert(1, "loc_in_doc", token_loc_in_doc)
    df = df.explode(["loc_in_doc", "tokens", "labels"])
    df = df.sort_values(["document", "loc_in_doc"]).reset_index(drop=True)
    return df


In [9]:
df = read_tokens_df()
df["labels"].value_counts()

Filtered 115 chunks


labels
O                     807885
I-EMAIL                66774
I-FULLNAME             57841
I-NAME                 57486
B-FULLNAME             20187
                       ...  
I-JOBTYPE                 13
B-PIN                     11
I-SECONDARYADDRESS        10
B-ORDINALDIRECTION         8
B-SECONDARYADDRESS         4
Name: count, Length: 93, dtype: int64